In [23]:
from ltn_imp.automation.knowledge_base import KnowledgeBase
import torch
import pandas as pd

In [24]:
!poetry run poe download-medical-datasets

Poe => mkdir -p examples/medical/datasets
Poe => curl -L -o examples/medical/datasets/pima_indians_imputed.csv https://raw.githubusercontent.com/ChristelSirocchi/hybrid-ML/main/pima_indians_imputed.csv
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 33428  100 33428    0     0   581k      0 --:--:-- --:--:-- --:--:--  582k


In [25]:
import random
import numpy as np

seed = 1024

# Set the random seed for reproducibility
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [26]:
from sklearn.model_selection import train_test_split

test_data = pd.read_csv('datasets/pima_indians_imputed.csv', index_col=0).astype(float)

y = test_data.iloc[:, -1]

x_train, x_test = train_test_split(test_data, test_size=0.5, random_state=seed, stratify=y)
y_train = x_train.iloc[:, -1]  # Extract labels from the training split
x_train, x_val = train_test_split(x_train, test_size=0.1, random_state=seed, stratify=y_train)

x_train.to_csv('datasets/train.csv')
x_val.to_csv('datasets/val.csv')
x_test.to_csv('datasets/test.csv')

In [27]:
import torch
from sklearn.metrics import matthews_corrcoef

def predict(model, x):
    model.eval()  # Ensure the model is in evaluation mode
    with torch.no_grad():  # No need to track gradients
        # Ensure x is a tensor and has the right dtype
        if not isinstance(x, torch.Tensor):
            x = torch.tensor(x, dtype=torch.float32)
        elif x.dtype != torch.float32:
            x = x.float()
        
        # Forward pass through the model
        probs = model(x)
        
        # Apply binary classification threshold at 0.5
        preds = (probs > 0.5).float()
    return preds

def compute_metrics(model, data_loader):
    correct = 0
    total = 0
    
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    true_negatives = 0
    
    all_true_labels = []
    all_predicted_labels = []
    
    with torch.no_grad():  # Disable gradient computation
        for data, labels in data_loader:
            # Ensure data and labels are the correct dtype
            if not isinstance(data, torch.Tensor):
                data = torch.tensor(data, dtype=torch.float32)
            elif data.dtype != torch.float32:
                data = data.float()
            
            if not isinstance(labels, torch.Tensor):
                labels = torch.tensor(labels, dtype=torch.float32)
            elif labels.dtype != torch.float32:
                labels = labels.float()
            
            # Get predictions
            preds = predict(model, data)
            
            # Squeeze predictions and labels to remove dimensions of size 1
            predicted_labels = preds.squeeze()
            true_labels = labels.squeeze()

            # Ensure the shapes match before comparison
            if predicted_labels.shape != true_labels.shape:
                true_labels = true_labels.view_as(predicted_labels)
            
            # Collect all predictions and true labels for MCC
            all_true_labels.extend(true_labels.cpu().numpy())
            all_predicted_labels.extend(predicted_labels.cpu().numpy())

            # Count correct predictions
            correct += (predicted_labels == true_labels).sum().item()
            total += true_labels.size(0)
            
            # Calculate TP, FP, FN, TN
            true_positives += ((predicted_labels == 1) & (true_labels == 1)).sum().item()
            false_positives += ((predicted_labels == 1) & (true_labels == 0)).sum().item()
            false_negatives += ((predicted_labels == 0) & (true_labels == 1)).sum().item()
            true_negatives += ((predicted_labels == 0) & (true_labels == 0)).sum().item()
    
    accuracy = correct / total if total > 0 else 0
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    balanced_accuracy = 0.5 * (recall + (true_negatives / (true_negatives + false_positives) if (true_negatives + false_positives) > 0 else 0))
    mcc = matthews_corrcoef(all_true_labels, all_predicted_labels)
    tnr = true_negatives / (true_negatives + false_positives) if (true_negatives + false_positives) > 0 else 0  # True Negative Rate
    fpr = false_positives / (false_positives + true_negatives) if (false_positives + true_negatives) > 0 else 0  # False Positive Rate
    fnr = false_negatives / (false_negatives + true_positives) if (false_negatives + true_positives) > 0 else 0  # False Negative Rate
    tpr = recall  # True Positive Rate is the same as recall

    print(f"True Positives: {true_positives}, False Positives: {false_positives}, False Negatives: {false_negatives}, True Negatives: {true_negatives}, Total: {total}")
    print()
    print(f"A (Accuracy): {accuracy:.4f}")
    print(f"P (Precision): {precision:.4f}")
    print(f"R (Recall): {recall:.4f}")
    print(f"F1 (F1 Score): {f1_score:.4f}")
    print(f"BA (Balanced Accuracy): {balanced_accuracy:.4f}")
    print(f"MCC (Matthews Correlation Coefficient): {mcc:.4f}")
    print(f"TNR (True Negative Rate): {tnr:.4f}")
    print(f"FPR (False Positive Rate): {fpr:.4f}")
    print(f"FNR (False Negative Rate): {fnr:.4f}")
    print(f"TPR (True Positive Rate): {tpr:.4f}")
    model.train()

In [28]:
kb = KnowledgeBase("medical_config.yaml", device="cuda")

Using device: cpu


In [29]:
compute_metrics(kb.predicates["Diabetic"], kb.loaders[0])

True Positives: 1, False Positives: 16, False Negatives: 119, True Negatives: 209, Total: 345

A (Accuracy): 0.6087
P (Precision): 0.0588
R (Recall): 0.0083
F1 (F1 Score): 0.0146
BA (Balanced Accuracy): 0.4686
MCC (Matthews Correlation Coefficient): -0.1381
TNR (True Negative Rate): 0.9289
FPR (False Positive Rate): 0.0711
FNR (False Negative Rate): 0.9917
TPR (True Positive Rate): 0.0083


In [30]:
compute_metrics(kb.predicates["Diabetic"], kb.test_loaders[0])

True Positives: 1, False Positives: 20, False Negatives: 133, True Negatives: 230, Total: 384

A (Accuracy): 0.6016
P (Precision): 0.0476
R (Recall): 0.0075
F1 (F1 Score): 0.0129
BA (Balanced Accuracy): 0.4637
MCC (Matthews Correlation Coefficient): -0.1521
TNR (True Negative Rate): 0.9200
FPR (False Positive Rate): 0.0800
FNR (False Negative Rate): 0.9925
TPR (True Positive Rate): 0.0075


In [31]:
kb.optimize(num_epochs=201, log_steps=100, lr=0.0001, early_stopping=True, patience=5)

['∀ person.(((y == diabetes) -> Diabetic(person)))', '∀ person.(((y == healthy) -> ~(Diabetic(person))))', '∀ person.((((person[BMI] > 29) & (person[Glucose] > 125)) -> Diabetic(person)))', '∀ person.((((person[BMI] < 26) & (person[Glucose] < 101)) -> ~(Diabetic(person))))']
Rule Outputs:  [tensor(0.9214, grad_fn=<RsubBackward1>), tensor(0.7023, grad_fn=<RsubBackward1>), tensor(0.9453, grad_fn=<RsubBackward1>), tensor(0.9500, grad_fn=<RsubBackward1>)]
Epoch 1/201, Train Loss: 0.15833216905593872, Validation Loss: 0.196571946144104

Rule Outputs:  [tensor(0.9399, grad_fn=<RsubBackward1>), tensor(0.8606, grad_fn=<RsubBackward1>), tensor(0.8764, grad_fn=<RsubBackward1>), tensor(0.9852, grad_fn=<RsubBackward1>)]
Early stopping at Epoch 7/201, Train Loss: 0.09816032648086548, Validation Loss: 0.19458556175231934


In [32]:
compute_metrics(kb.predicates["Diabetic"], kb.loaders[0])

True Positives: 70, False Positives: 40, False Negatives: 50, True Negatives: 185, Total: 345

A (Accuracy): 0.7391
P (Precision): 0.6364
R (Recall): 0.5833
F1 (F1 Score): 0.6087
BA (Balanced Accuracy): 0.7028
MCC (Matthews Correlation Coefficient): 0.4145
TNR (True Negative Rate): 0.8222
FPR (False Positive Rate): 0.1778
FNR (False Negative Rate): 0.4167
TPR (True Positive Rate): 0.5833


In [33]:
compute_metrics(kb.predicates["Diabetic"], kb.test_loaders[0])

True Positives: 86, False Positives: 57, False Negatives: 48, True Negatives: 193, Total: 384

A (Accuracy): 0.7266
P (Precision): 0.6014
R (Recall): 0.6418
F1 (F1 Score): 0.6209
BA (Balanced Accuracy): 0.7069
MCC (Matthews Correlation Coefficient): 0.4080
TNR (True Negative Rate): 0.7720
FPR (False Positive Rate): 0.2280
FNR (False Negative Rate): 0.3582
TPR (True Positive Rate): 0.6418
